In [2]:
import numpy as np
import math
import os
from bnn_priors.models.conv_nets import ClassificationConvNet, CorrelatedClassificationConvNet
from bnn_priors import exp_utils

In [3]:
data = exp_utils.get_data("mnist", "cpu")

In [4]:
x_train = data.norm.train_X
y_train = data.norm.train_y

In [5]:
width = 50
depth = 3
weight_prior = exp_utils.get_prior("gaussian")
bias_prior = exp_utils.get_prior("gaussian")
weight_loc = 0.
weight_scale = 2.**0.5
bias_loc = 0.
bias_scale = 1.
scaling_fn = lambda std, dim: std/dim**0.5
weight_prior_params = {}
bias_prior_params = {}

In [6]:
if len(x_train.shape) == 4:
    in_channels = x_train.shape[1]
    img_height = x_train.shape[-2]
else:
    in_channels = 1
    img_height = int(math.sqrt(x_train.shape[-1]))
net = CorrelatedClassificationConvNet(in_channels, img_height, y_train.max()+1, width, depth, softmax_temp=1.,
               prior_w=weight_prior, loc_w=weight_loc, std_w=weight_scale,
               prior_b=bias_prior, loc_b=bias_loc, std_b=bias_scale, scaling_fn=scaling_fn,
               weight_prior_params=weight_prior_params, bias_prior_params=bias_prior_params).to(x_train)

In [8]:
list(net.modules())[-9].weight_prior

ConvCorrelatedNormal()

In [7]:
wp = weight_prior((50, 50, 3, 3), weight_loc, weight_scale)

In [8]:
wp.log_prob()

tensor(-39558.5469, grad_fn=<SumBackward0>)

In [9]:
p = exp_utils.get_prior("convcorrnormal")

In [13]:
wp = p((50, 50, 3, 3), weight_loc, weight_scale, lengthscale=1e-6)

In [14]:
wp.log_prob()

torch.Size([50, 50])


tensor(-39782.1289, grad_fn=<SumBackward0>)

In [12]:
wp._dist_obj().scale_tril

tensor([[1.4142, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.4142, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.4142, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.4142, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.4142, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.4142, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.4142, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.4142, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.4142]])

In [13]:
weight_scale

1.4142135623730951

In [14]:
weight_loc

0.0